In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare

# Chi Square Test

I suspect the dice in my Settlers of Catan set are not fair dice. So I wrote down all the outcomes of two games and I'm going to do a Chi Square test for goodness of fit.

[TL;DR](#end)

# Load the data and make it pretty

In [2]:
raw = pd.read_csv('../data/catan_dice.csv')
raw.head(1)

,Value,Count G1,Count G2
0,2,2,1


In [3]:
df = pd.DataFrame(
    raw[['Count G1', 'Count G2']].values,
    index=raw['Value'],
    columns=['G1', 'G2']
)

df.head(1)

,G1,G2
Value,,
2,2,1


# Roll outcomes for each game and sum of both

In [4]:
df['Both'] = df['G1'] + df['G2']
df

,G1,G2,Both
Value,,,
2,2,1,3
3,2,2,4
4,5,0,5
5,5,10,15
6,13,7,20
7,6,3,9
8,6,5,11
9,4,1,5
10,7,2,9


# Get expected distribution for the sum of two dice

In [5]:
from collections import defaultdict

In [6]:
e = defaultdict(float)

for i in range(1,7):
    for j in range(1,7):
        e[i+j] += 1/36

expected = pd.Series(e)

print('Probability of the sum of two dice')
expected

Probability of the sum of two dice


2     0.027778
3     0.055556
4     0.083333
5     0.111111
6     0.138889
7     0.166667
8     0.138889
9     0.111111
10    0.083333
11    0.055556
12    0.027778
dtype: float64

# Find expected outcomes based on total rolls

In [7]:
xdf = pd.DataFrame(index=df.index)

for col in df.columns:
    
    total_observations = df[col].sum()
    xdf[col] = total_observations * expected.values
    
    print(f'{col: <4} had {total_observations} observations.')

xdf

G1   had 52 observations.
G2   had 31 observations.
Both had 83 observations.


,G1,G2,Both
Value,,,
2,1.444444,0.861111,2.305556
3,2.888889,1.722222,4.611111
4,4.333333,2.583333,6.916667
5,5.777778,3.444444,9.222222
6,7.222222,4.305556,11.527778
7,8.666667,5.166667,13.833333
8,7.222222,4.305556,11.527778
9,5.777778,3.444444,9.222222
10,4.333333,2.583333,6.916667


## Chi Square test

    chisq, p = scipy.stats.chisquare(f_obs, f_exp=None, ddof=0, axis=0)

[Citation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html)

In [8]:
for col in df.columns:
    
    oe = pd.DataFrame({
        'Observed': df[col],
        'Expected': xdf[col]
    })
    
    chisq, p = chisquare(oe['Observed'], oe['Expected'], ddof=1)
    
    w = 10  # For formatting purposes
    print(
        f'\n{"":=>30}\n'
        
        f'{"Group:": <{w}}{col}\n'
        f'{"Chisq:": <{w}}{chisq:.4}\n'
        f'{"P:": <{w}}{p:.4}\n'        
        f'{oe}\n'
        
        f'{"":=>30}\n')


Group:    G1
Chisq:    9.904
P:        0.3583
       Expected  Observed
Value                    
2      1.444444         2
3      2.888889         2
4      4.333333         5
5      5.777778         5
6      7.222222        13
7      8.666667         6
8      7.222222         6
9      5.777778         4
10     4.333333         7
11     2.888889         1
12     1.444444         1


Group:    G2
Chisq:    22.28
P:        0.008022
       Expected  Observed
Value                    
2      0.861111         1
3      1.722222         2
4      2.583333         0
5      3.444444        10
6      4.305556         7
7      5.166667         3
8      4.305556         5
9      3.444444         1
10     2.583333         2
11     1.722222         0
12     0.861111         0


Group:    Both
Chisq:    18.51
P:        0.02971
        Expected  Observed
Value                     
2       2.305556         3
3       4.611111         4
4       6.916667         5
5       9.222222        15
6      11.5277

# Conclusion <a class="anchor" id="end" />

I don't like the look of this. Game one was ok. But Game two was absolutely a fluke! (If we assume the dice are fair.) But let's say there were too few rolls in game two. When we combine the games we still get something I don't quite like.

If we assume the dice are fair, we have a 2% chance that the observed distribution is at least this far from the expected distribution. Ok 2% is not so bad... but I lost game two because all of my opponents were on hexes with 5s on them and they all did better than me so... I conclude that the game was rigged in their favor.